In [1]:
!echo "spark.driver.extraClassPath=file:///opt/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar" >> /etc/spark/conf/spark-defaults.conf
!echo "spark.executor.extraClassPath=file:///opt/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar" >> /etc/spark/conf/spark-defaults.conf

In [ ]:
!hadoop fs -mkdir /user/livy

In [ ]:
val scaleFactor = "1"           // data scale 1GB
val iterations = 1              // how many times to run the whole set of queries.
val format = "parquet"          // support parquet or orc
val storage = "gs"              // support Google Cloud Storage buckets -- "gs", or HDFS -- "hdfs"
var bucket_name = "oap-dev"     // Please change to your bucket name 
val partitionTables = true      // create partition tables
val query_filter = Seq()        // Seq() == all queries
//val query_filter = Seq("q1-v2.4", "q2-v2.4") // run subset of queries
val randomizeQueries = false    // run queries in a random order. Recommended for parallel runs.

if (storage == "hdfs"){
    bucket_name = "/user/livy"  // scala notebook only has the write permission of "hdfs:///user/livy" directory       
}

// detailed results will be written as JSON to this location.
var resultLocation = s"${storage}://${bucket_name}/results/tpcds_${format}/${scaleFactor}/"
var databaseName = s"tpcds_${format}_scale_${scaleFactor}_db"
val use_arrow = false            // when you want to use gazella_plugin to run TPC-DS, you need to set it true.

if (use_arrow){
    val data_path= s"${storage}://${bucket_name}/datagen/tpcds_${format}/${scaleFactor}"
    resultLocation = s"${storage}://${bucket_name}/results/tpcds_arrow/${scaleFactor}/"
    databaseName = s"tpcds_arrow_scale_${scaleFactor}_db"
    val tables = Seq("call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site")
    sql(s"DROP database $databaseName CASCADE")
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"$databaseName has exists!")
    }else{
        spark.sql(s"create database if not exists $databaseName").show
        spark.sql(s"use $databaseName").show
        for (table <- tables) {
            if (spark.catalog.tableExists(s"$table")){
                println(s"$table has exists!")
            }else{
                spark.catalog.createTable(s"$table", s"$data_path/$table", "arrow")
            }
        }
        if (partitionTables) {
            for (table <- tables) {
                try{
                    spark.sql(s"ALTER TABLE $table RECOVER PARTITIONS").show
                }catch{
                        case e: Exception => println(e)
                }
            }
        }
    }
}


In [ ]:
val timeout = 60 // timeout in hours

// COMMAND ----------

// Spark configuration
spark.conf.set("spark.sql.broadcastTimeout", "10000") // good idea for Q14, Q88.

// ... + any other configuration tuning

// COMMAND ----------
sql(s"use $databaseName")
import com.databricks.spark.sql.perf.tpcds.TPCDS
val tpcds = new TPCDS (sqlContext = spark.sqlContext)
def queries = {
  val filtered_queries = query_filter match {
    case Seq() => tpcds.tpcds2_4Queries
    case _ => tpcds.tpcds2_4Queries.filter(q => query_filter.contains(q.name))
  }
  if (randomizeQueries) scala.util.Random.shuffle(filtered_queries) else filtered_queries
}
val experiment = tpcds.runExperiment(
  queries, 
  iterations = iterations,
  resultLocation = resultLocation,
  tags = Map("runtype" -> "benchmark", "database" -> databaseName, "scale_factor" -> scaleFactor))

println(experiment.toString)
experiment.waitForFinish(timeout*60*60)